<H2>1. 必要なライブラリを読み込む</H2>

In [ ]:
% matplotlib inline

from __future__ import print_function

import copy

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

matplotlib.style.use('ggplot')


<H2>2. データの読み込み</H2>
<p> pandasにはread_csvという関数が用意されており、それを用いれば簡単にCSVファイルを読み込むことができます</p>
<p>データはdatasetフォルダ直下にある"kc_house_data.csv"を使用します。</p>

In [ ]:
dataset = pd.read_csv('dataset/kc_house_data.csv')

In [ ]:
dataset.head()

In [ ]:
print('Number of Rows: %i   Number of Columns: %i' % dataset.shape)

<h2>3. 要約統計量を出力する</h2>
<p> describeでデータ数、平均や中央値、標準偏差などの統計量が出力される</p>

In [ ]:
dataset.describe()

<H2>4. 最も基本的なデータの操作</H2>

<h3> インデックスを用いた行の指定</h3>
<p>インデックス番号0~10の行を抽出する</p>

In [ ]:
dataset.ix[0:10]

<h3> カラム名を用いた列の指定</h3>
<p> 最初の10行の"price"と"bedrooms"の列を抽出する</p>

In [ ]:
dataset.ix[:9][['price', 'bedrooms']]
# dataset.priceでも良い

<h3>カラム間の演算</h3>
<p>ここでは例として"sqft_above"と"sqft_basement"を合計した"sqft_total"を新たな列として加える</p>

In [ ]:
dataset['sqft_total'] = dataset['sqft_above'] + dataset['sqft_basement']
dataset.ix[0:9][['id', 'sqft_above', 'sqft_basement', 'sqft_total']]

<h3>カラムに関数を適用する</h3>
<p>"date"は文字列になっているが、これを年と月で分解して新しいカラムとして追加する</p>

In [ ]:
def date_str2year(x):
    # 最初の４文字を取り出せば、年になる
    return int(x[:4])

def date_str2month(x):
    # 5文字から6文字目を取り出せば、月になる
    return int(x[4:6])

In [ ]:
dataset['date_year'] = dataset['date'].apply(date_str2year)
dataset['date_month'] = dataset['date'].apply(date_str2month)
dataset.ix[0:9][['id','date', 'date_year', 'date_month']]

<h3>ダミー変数を作る</h3>
<p> 一般的にはカテゴリ変数を機械学習のモデルに投入する際、0と1のダミー変数に変換する。</p>

In [ ]:
dataset = pd.get_dummies(data=dataset, columns=['view'])

<p>上記を実行すると、"view"という変数の代わりに、"view_1", "view_2"..."vier4"というダミー変数に展開される。</p>

In [ ]:
dataset.columns

In [ ]:
dataset.head()

<h3>条件でのフィルタリング</h3>
<p>bedroomsが1以下の物件のみを抽出する</p>

<p>(参考)pandasのデータフレームへのカラムへのアクセスですが、これまでdataset["カラム名"]という書き方をしていましたが、別の方法としてdataset.カラム名というアクセスの仕方があります。</p>

In [ ]:
dataset[dataset['bedrooms'] < 1]
# 以下と同じ
# dataset[dataset.bedrooms < 1]

<h3>複数条件でのデータ抽出</h3>
<p>ここではbedroomsもbathroomsも1以下の物件を抽出</p>

<p>numpyの関数logical_andを使って指定するやり方</p>

In [ ]:
dataset[np.logical_and((dataset['bedrooms'] < 1), (dataset['bathrooms'] < 1))]

<p>numpyのallを使って指定するやり方</p>

In [ ]:
a = np.array(dataset['bedrooms'] < 1)
b = np.array(dataset['bathrooms'] < 1)

In [ ]:
dataset[np.all([a, b], axis=0)]

<p>ビット演算子の"&"を使って指定するやり方。ただし、ビット演算子なので、"(dataset["bedrooms"] <1)" は必ずTrue/Falseになるので問題ないが、それ以外の値が来た場合、予期せぬ動きをする可能性があるので注意が必要</p>

In [ ]:
dataset[(dataset['bedrooms'] < 1) & (dataset['bathrooms'] < 1)]

<h2>5. 分布の確認(データ可視化入門）</h2>

<h3>ヒストグラム</h3>
<p>ひとつの数値データのバラツキを可視化する際にはヒストグラムが有効</p>

In [ ]:
dataset['price'].hist()
# 以下でも同じ
# dataset.price.hist()

<p>logスケールにしたい場合は、Numpyのlog関数を使って行う</p>

In [ ]:
plt.hist(np.log(dataset['price']))

<h3>散布図</h3>
<p>2つの変数のバラツキや相関関係を確認するには散布図を使う</p>

In [ ]:
dataset.plot(kind='scatter', x='sqft_lot', y='price')

<h3>Group Byで集計して棒グラフ</h3>
<p> "condition"はカテゴリ変数なので、この変数をキーにして"price"の平均を集計し、それを棒グラフにします</p>

In [ ]:
price_by_condition = \
 dataset.groupby('condition').aggregate({'price': np.mean}).reset_index()
price_by_condition.plot.bar(x='condition')

<h3>箱ヒゲ図(Boxplot) </h3>
<p>棒グラフにすると平均の比較はできますが、分布全体の比較はできません。そこで箱ヒゲ図の出番です。</p>
<p>各要因ごとに分布を比較したいときに箱ヒゲ図はとても便利です</p>

In [ ]:
dataset[['condition', 'price']].boxplot(by='condition')

<h2>6. 欠損の確認とその対応</h2>
<p> 欠損値がある場合、本来はその発生原因を理解する必要があります。なぜなら、その原因によって対応方針は異なるためです。</p>
<p>ここでは、欠損があるカラムをチェックします</p>
<br>
<p>まずあまり効率的ではないものの、pandasに慣れるため各列ごとにチェックしていきます</p>

In [ ]:
col_names = dataset.columns
for col_name in col_names:
    missing_num = sum(pd.isnull(dataset[col_name]))
    print(col_name, '; # of missing record: ', missing_num)

<p>applyを使って欠損を確認する方法</p>

In [ ]:
missing_check = dataset.copy()
missing_check = missing_check.apply(pd.isnull, axis=1)
missing_check.apply(sum, axis=0)

<p>欠損はない。もしあれば、dataset.fillna()を使って補完するか、dataset.dropna()で欠損があるレコードを削除する</p>

<h2>7. 特徴量の作成</h2>
<p> 既存の特徴量を使って（ターゲットの変数である"price"を除いて）新しい特徴量を作りましょう</p>

In [ ]:
dataset['sqft_living_div_sqft_living15'] = \
 dataset['sqft_living'] / (dataset['sqft_living15'] + 0.001)

In [ ]:
'Enter your code here'

<h2>8. Scikit Learnを用いた予測モデルの構築</h2>

<p>早速、機械学習モデルを使ってみよう</p>
<p>ここでは以下の機械学習アルゴリズムを試します</p>
<li>線形回帰モデル</li>
<li>決定木</li>
<li>ランダムフォレスト</li>
<p>また、モデル評価はMSEとします</p>

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import train_test_split
from sklearn.metrics import mean_squared_error

<h3>ターゲット変数と特徴量を指定してsklearnに渡せるように準備する</h3>

In [ ]:
target_col = 'price'
exclude_cols = ['price', 'id', 'date']
feature_cols = [col for col in dataset.columns if col not in exclude_cols]

In [ ]:
feature_cols

In [ ]:
y = np.array(dataset[target_col])
X = np.array(dataset[feature_cols])

In [ ]:
X_train, X_test, y_train, y_test = \
 train_test_split(X, y, test_size=0.3, random_state=1234)
X_train1, X_train2, y_train1, y_train2 = \
 train_test_split(X_train, y_train, test_size=0.3, random_state=1234)

<h2>8-1. アルゴリズムの選択</h2>

<h3>線形回帰モデル</h3>

In [ ]:
lm = LinearRegression()
lm.fit(X_train1, y_train1)
y_pred2 = lm.predict(X_train2)
lm_mse = mean_squared_error(y_train2, y_pred2)
print('LinerRegression MSE: ', lm_mse)

<h3>回帰木</h3>

In [ ]:
dt = DecisionTreeRegressor()
dt.fit(X_train1, y_train1)
y_pred2 = dt.predict(X_train2)
dt_mse = mean_squared_error(y_train2, y_pred2)
print('DT MSE: ', dt_mse)

<h3>ランダムフォレスト</h3>

In [ ]:
rf = RandomForestRegressor(random_state=1234)
rf.fit(X_train1, y_train1)
y_pred2 = rf.predict(X_train2)
rf_mse = mean_squared_error(y_train2, y_pred2)
print('RandomForest MSE: ', rf_mse)

In [ ]:
algos = ['LinerRegression', 'DecisionTree', 'RandomForest']
mses = [lm_mse,dt_mse, rf_mse]
print('Best Alogorithms : ', algos[np.argmin(mses)] )

<h2>8-2. 変数選択</h2>
<h3>変数増加法で変数選択をしてみる</h3>

In [ ]:
def get_gfs_feature_indices(X, y, features, clf):
    X_train_, X_test_, y_train_, y_test_ = \
        train_test_split(X, y, test_size=0.3, random_state=1234)
    feature_indices = {feature: idx for idx, feature in enumerate(features)}
    features = set(features)
    last_mse = np.inf
    chosen_features = set()
    while len(chosen_features) < len(features):
        mse_features = []
        for feature in (features - chosen_features):
            candidates = chosen_features.union(set([feature]))
            indices = [feature_indices[feature] for feature in candidates]
            clf.fit(X_train_[:, indices], y_train_)
            y_pred = clf.predict(X_test_[:, indices])
            mse = mean_squared_error(y_test_, y_pred)
            mse_features += [(mse, feature)]
        mse, feature = min(mse_features)
        if mse >= last_mse:
            break
        last_mse = mse
        print('Newly Added Feature: {},\tMSE Score: {}'.format(feature, mse))
        chosen_features.add(feature)
    return [feature_indices[feature] for feature in chosen_features]


In [ ]:
selected_feature_index = \
get_gfs_feature_indices(X=X_train, 
                        y=y_train, 
                        features=feature_cols, 
                        clf=RandomForestRegressor(random_state=1234))

<h3>改めて、変数選択後の精度を確認してみましょう</h3>

In [ ]:
rf = RandomForestRegressor(random_state=1234)
rf.fit(X_train1[:, selected_feature_index], y_train1)
y_pred2 = rf.predict(X_train2[:, selected_feature_index])
rf_mse = mean_squared_error(y_train2, y_pred2)
print('RandomForest MSE: ', rf_mse)

<h2>8-3. クロスバリデーションによるパラメーターのチューニング</h2>

In [ ]:
rf = RandomForestRegressor(random_state=1234)

In [ ]:
params = {'n_estimators': [10, 50, 100], 'max_depth': [5, 10, 50]}

In [ ]:
import sklearn 
sklearn.__version__

In [ ]:
#注意: scikit-learnのバージョンが、0.18の場合は、scoring='neg_mean_squared_error'とします
# gscv = GridSearchCV(rf, param_grid=params, verbose=1,
#                     cv=3, scoring='neg_mean_squared_error')

#注意: scikit-learnのバージョンが、0.17の場合は、scoring='mean_squared_error'とします
gscv = GridSearchCV(rf, param_grid=params, verbose=1,
                    cv=3, scoring='mean_squared_error')

In [ ]:
gscv.fit(X_train1[:, selected_feature_index], y_train1)

In [ ]:
gscv.best_params_

<h3>パラメーターチューニング後のスコアを見てみましょう</h3>

In [ ]:
rf = RandomForestRegressor(n_estimators=100, max_depth=50, random_state=1234)
rf.fit(X_train1[:, selected_feature_index ], y_train1)
y_pred2 = rf.predict(X_train2[:, selected_feature_index])
rf_mse = mean_squared_error(y_train2, y_pred2)
print('RandomForest MSE: ', rf_mse)

<h2>8-4. テストデータへ適用して精度を確認する</h2>

<h3>モデルの学習</h3>

In [ ]:
rf = RandomForestRegressor(n_estimators=100, max_depth=50, random_state=1234)
rf.fit(X_train[:, selected_feature_index], y_train)

In [ ]:
y_pred_on_test = rf.predict(X_test[:, selected_feature_index])
rf_mse = mean_squared_error(y_test, y_pred_on_test)
print('RandomForest MSE: ', rf_mse)

<h2>8-5. 誤差の様子を可視化して確認</h2>

<h3>予測数値と実際の値の散布図</h3>

In [ ]:
plt.scatter(y_test, y_pred_on_test)

<h3>誤差のヒストグラム</h3>

In [ ]:
error_rate = (y_test - y_pred_on_test) / y_test
plt.hist(error_rate)
print('Mean: ', np.mean(error_rate))
print('Std: ', np.std(error_rate))